# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [7]:
email='enron_mails_small.csv'

In [8]:
emails=sc.textFile(email).cache()
list(enumerate(emails.first().split(",")))

[(0, 'Date'), (1, 'From'), (2, 'To')]

In [14]:
def findmail(x):
    row = []
    for string in x.replace('@enron.com','').split('.'):
        row.append(string.capitalize())
    return " ".join(row)

def extractdata(pid,rows):
    if pid == 0:
        next(rows)
    import csv
    for count in csv.reader(rows):
        if '@enron.com' not in count[1] :
            continue
        from_ = findmail(count[1])
        for x in count[2].split(';'):
            if '@enron.com' in x :
                yield (from_,findmail(x))
                

def reducer1(count):
    return (count[1],count[0])

def filterkey(count):
    if count[0]>=count[1]:
        return ((count[1],count[0]),1)
    if count[0]<count[1]:
        return ((count[0],count[1]),1)
    
def combine(count):
    return ('recipropcal',(" : ").join(count))
        
        
rdd = emails.mapPartitionsWithIndex(extractdata).distinct()
rdd2 = rdd.map(reducer1)
result = rdd2.intersection(rdd).map(filterkey).distinct()\
            .sortByKey().filter(lambda x : x[0][0] !=x[0][1]).keys().map(combine)
result.take(40)
    

[('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Carol Clair : Richard Sanders'),
 ('recipropcal', 'Carol Clair : Sara Shackleton'),
 ('recipropcal', 'Carol Clair : Tana Jones'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
 ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
 ('recipropcal', 'Eric Bass : Susan Scott'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Mark Taylor'),
 ('recipropcal', 'Janette El

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [12]:
import pyspark

In [13]:
SAT_FN='SAT_Results.csv '
HSD_FN='DOE_High_School_Directory_2014-2015.csv '

In [15]:
sat=sc.textFile(SAT_FN).cache()
list(enumerate(sat.first().split(",")))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [72]:
def extractScore(pid, rows):
    if pid == 0:
        next(rows)
    import csv
    for fields in csv.reader(rows):
        if fields[2]!='s': 
            yield(fields[0],(int(fields[2]),int(fields[4])))

            #yield(fields[0],(int(fields[2]),int(fields[4])*int(fields[2])))

satScores = sat.mapPartitionsWithIndex(extractScore)
satScores.take(5)

[('02M047', (16, 400)),
 ('21K410', (475, 437)),
 ('30Q301', (98, 440)),
 ('17K382', (59, 374)),
 ('18K637', (35, 381))]

In [53]:
schools = sc.textFile(HSD_FN).cache()
#list(enumerate(schools.first().split(",")))

In [35]:
def extractbus(pid, rows):
    if pid == 0:
        next(rows)
    import csv
    for fields in csv.reader(rows):
        if len(fields)==58:
             for bus_line in fields[10].split(','):    
                yield(fields[0],bus_line)
schools_bus = schools.mapPartitionsWithIndex(extractbus) \
                .map(lambda x: (x[0],x[1]))

schools_bus.take(5)

[('01M292', 'B39'),
 ('01M292', ' M14A'),
 ('01M292', ' M14D'),
 ('01M292', ' M15'),
 ('01M292', ' M15-SBS')]

In [56]:
def extractsubway(pid, rows):
    if pid == 0:
        next(rows)
    import csv
    for fields in csv.reader(rows):
        if len(fields)==58:
             for subway_line in fields[11].split(','):    
                if len(subway_line.split(';')) > 1:
                    for s_line in subway_line.split(';'):
                        yield(fields[0],s_line)
                else:
                    yield(fields[0],subway_line)
schools_subway = schools.mapPartitionsWithIndex(extractsubway) \
                .map(lambda x: (x[0],x[1]))

schools_subway.take(5)

[('01M292', 'B'),
 ('01M292', ' D to Grand St '),
 ('01M292', ' F to East Broadway '),
 ('01M292', ' J'),
 ('01M292', ' M')]

In [49]:
df = spark.read.load(HSD_FN, format= 'csv',header = True, inferSchema = True)
df = df.filter(df['total_students']>500)\
        .select('dbn','bus','subway')

df.show()

+------+--------------------+--------------------+
|   dbn|                 bus|              subway|
+------+--------------------+--------------------+
|01M450|M101, M102, M103,...|6 to Astor Place ...|
|01M539|B39, M14A, M14D, ...|F, J, M, Z to Del...|
|01M696|M14A, M14D, M21, ...|                 N/A|
|02M374|M101, M102, M103,...|1, 2, 3, F, M to ...|
|02M400|M10, M104, M11, M...|1, A, B, C, D to ...|
|02M408|M10, M104, M11, M...|1, C, E to 50th S...|
|02M412|M11, M14A, M14D, ...|1 to 18th St ; 2,...|
|02M413|M101, M102, M14A,...|4, 5, L, Q to 14t...|
|02M416|M101, M102, M15, ...|        6 to 77th St|
|02M418|M15, M15-SBS, M20...|1, R to Whitehall...|
|02M420|M101, M102, M103,...|4, 5, 6, N, Q, R ...|
|02M425|M103, M15, M15-SB...|1, R to Rector St...|
|02M475|M20, M22, M5, M9,...|1, 2, 3, A, C, E ...|
|02M489|M103, M15, M15-SB...|1 to Rector St ; ...|
|02M519|M101, M102, M15, ...|4, 5, N, Q, R to ...|
|02M520|M103, M15, M22, M...|1 to Chambers St ...|
|02M529|M104, M11, M2, M2...|1,

In [77]:
bus = schools_bus.join(satScores) \
            .values() \
            .mapValues(lambda x: (x[0], x[0]*x[1])) \
            .reduceByKey(lambda x,y: (x[0]+y[0], x[1] + y[1])) \
            .mapValues(lambda x: x[1]//x[0]) \
            .collect()

bus[:20]


[('M14A', 529),
 (' M22', 574),
 (' M14A', 468),
 (' M23', 462),
 (' M42', 410),
 (' M50', 438),
 (' M4', 442),
 ('M103', 453),
 (' M34-SBS', 417),
 (' M100', 427),
 (' M116', 467),
 (' Bx40', 400),
 (' Bx42', 400),
 (' Bx2', 470),
 (' Bx21', 389),
 (' Bx41', 388),
 (' Bx4A', 391),
 (' Bx6', 400),
 (' Bx13', 386),
 (' Bx18', 380)]

In [78]:
subway = schools_subway.join(satScores) \
            .values() \
            .mapValues(lambda x: (x[0], x[0]*x[1])) \
            .reduceByKey(lambda x,y: (x[0]+y[0], x[1] + y[1])) \
            .mapValues(lambda x: x[1]//x[0]) \
            .collect()

subway[:5]

[('F to East Broadway ', 423),
 (' M', 456),
 (' L to 1st Ave', 443),
 (' E to 50th St', 386),
 (' R to Whitehall St-South Ferry ', 471)]